In [40]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
from scipy import signal
import pyedflib
from pyedflib import highlevel
import os
import h5py

In [41]:
#Constants 
dataPath = r"C:\Users\Humperdink2\Documents\github\NAThacks\Data\files"
outPath = r"C:\Users\Humperdink2\Documents\github\NAThacks\spec_data\train"
h5Path = r"h5py/specs.h5"
numChannels = 64
labels = {
    "T0" : "rest",
    "T1" : ["leftHand", "bothHands"],
    "T2" : ["rightHand", "bothFeet"]
}
frequency = 160
time_period = 640
prc_overlap = .90
chunk_size = 2500

In [47]:
rest = []
leftHand = []
rightHand = []
bothHands = []
bothFeet = []

In [43]:
with h5py.File("h5py/specs.h5", "w") as specs:
    specs.create_dataset("rest",(chunk_size,81,31,64), maxshape=(None,81,31,64),compression="gzip")
    specs.create_dataset("leftHand",(chunk_size,81,31,64), maxshape=(None,81,31,64), compression="gzip")
    specs.create_dataset("rightHand",(chunk_size,81,31,64), maxshape=(None,81,31,64), compression="gzip")
    specs.create_dataset("bothHands",(chunk_size,81,31,64), maxshape=(None,81,31,64), compression="gzip")
    specs.create_dataset("bothFeet",(chunk_size,81,31,64), maxshape=(None,81,31,64), compression="gzip")

def storeData(data, dataset : str, size):
    with h5py.File(h5Path, "r+") as f:
        df = f[dataset]
        current_shape = df.shape
        df.resize((current_shape[0] + size, 81, 31, 64))
        df[current_shape[0]:, :] = data
        print(f"{dataset} adjusted to {current_shape[0] + size}")

In [ ]:
#Generating spectograms Images
person = 1
for folder in os.listdir(dataPath):
    for fileName in os.listdir(os.path.join(dataPath,folder)):

        #Skipping files .event files
        if fileName.find("event") != -1:
            continue

        #reading EDF file and extracting data
        filePath = os.path.join(dataPath,folder,fileName)
        annotations = ""
        file = ""

        #Getting the annotations and Data
        file = pyedflib.EdfReader(filePath)
        annotations = file.readAnnotations()
        file.close()

        signals, signal_headers, header = highlevel.read_edf(filePath)
        
        count = 0
        
        #Looping through the various 
        for i, period in enumerate(annotations[0]):
            #Getting signal data for 4 second period
            data = signals[:,int(annotations[0][i]):int(annotations[0][i]+time_period)]

            f, t, image = signal.spectrogram(data,frequency, nperseg = frequency, noverlap = frequency * prc_overlap)
            image = np.transpose(image,(1,2,0))

            #Assigning label to file
            label = annotations[2][i]

            #Categories of the different tests
            leftOrRight = ["03","04","07","08","11","12"]
            if label == "T1":
                if any(x == fileName[5:7] for x in leftOrRight):
                    leftHand.append(image)
                    if len(leftHand) == chunk_size:
                        storeData(leftHand,"leftHand", chunk_size)
                        leftHand = []
                else: 
                    bothHands.append(image)
                    if len(bothHands) == chunk_size:
                        storeData(bothHands,"bothHands", chunk_size)
                        bothHands = []
            elif label == "T2":
                if any(x == fileName[5:7] for x in leftOrRight):
                    rightHand.append(image)
                    if len(rightHand) == chunk_size:
                        storeData(rightHand,"rightHand", chunk_size)
                        rightHand = []
                else:
                    bothFeet.append(image)
                    if len(bothFeet) == chunk_size:
                        storeData(bothFeet,"bothFeet", chunk_size)
                        bothFeet = []
            else:
                rest.append(image)
                if len(rest) == chunk_size:
                        storeData(rest,"rest", chunk_size)
                        rest = []
            count += 1
    print(f"person {person} completed")
    person +=1

In [46]:
storeData(rest,"rest", len(rest))
rest = []
storeData(bothFeet,"bothFeet", len(bothFeet))
bothFeet = []
storeData(leftHand,"leftHand", len(leftHand))
leftHand = []
storeData(rightHand, "rightHand",len(rightHand))
rightHand = []
storeData(bothHands,"bothHands",len(bothHands))
bothHands = []

rest adjusted to 23736
bothFeet adjusted to 7144
leftHand adjusted to 7161
rightHand adjusted to 7109
bothHands adjusted to 7115
